In [1]:
import numpy as np
import random as rnd
import pandas as pd
from tqdm.autonotebook import tqdm

/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  after removing the cwd from sys.path.


In [3]:
class Salesman:
    def __init__(self, c: int, population: int, elite: float=.2, mutation: float=.2, generate: bool=True, start: int=0, path: str="cities_example.txt", verbose: bool=False):
        self.cities = c
        self.start = start 
        self.pop = population
        self.c = []
        self.elite, self.mutrate = elite, mutation
        self.verbose = verbose
        
        if generate:
            self.d = self.init_cities(2 * 5)
            np.savetxt(path, self.d, fmt="%i")
        else:
            try:
                self.d = np.loadtxt(path, dtype=int)
            except OSError:
                exit("Error: If you specify the path argument with generate=False, then you have to create a [path].txt file following cities_example.txt format. It should be a square matrix for distances between cities with zeroes on the diagonal.")
        
        if self.verbose:
            print("Initialized TSP with map:")
            print(self.d)
            print(f"Starting from town #{self.start}")
    
    
    def init_cities(self, maxd: int) -> np.ndarray:
        d = np.random.randint(-maxd, maxd, (self.cities, self.cities))
        d = np.abs((d + d.T) // 2) + 1
        np.fill_diagonal(d, 0)
        return d
    
    
    def generate_chromosome(self) -> list:
        to_visit = [v for v in range(self.d.shape[0]) if not v == self.start]
        rnd.shuffle(to_visit)
        return [v for v in to_visit]
    
    
    def initialization(self) -> None:
        self.c = [self.generate_chromosome() for _ in range(self.pop)]
    
    
    def distance(self, c: list) -> int:
        d = 0
        for i in range(len(c) - 1):
            d += self.d[c[i]][c[i + 1]]
        return d
    
    
    def selection(self, unique: bool=False) -> tuple:
        df = pd.DataFrame([{"Road": c, "Length": self.distance(c)} for c in self.c])
        # sort by road length to keep the best roads
        df = df.sort_values("Length")
        self.c = df.Road.values[:int(len(self.c) * self.elite)].tolist()
        if unique:
            solution = df.values[0]
            return tuple(solution)
    
    
    def breed(self) -> None:
        i, length = 0, len(self.c)
        # duplicates chromosomes
        while len(self.c) < self.pop:
            self.c.append(self.c[i % length])
            i += 1
    
    
    def mutate(self, c: list) -> list:
        if rnd.random() < self.mutrate:
            a, b = rnd.randint(0, len(c) - 1), rnd.randint(0, len(c) - 1)
            tmp = c[a]
            c[a] = c[b]
            c[b] = tmp
        return c
    
    
    def mutations(self) -> None:
        self.c = [self.mutate(c) for c in self.c]
    
    
    def run(self, generations: int) -> tuple:
        self.initialization()
        for i in range(generations):
            self.selection()
            self.breed()
            if self.verbose:
                print(i, *self.c, sep="\n")
            self.mutations()
        return self.selection(unique=True)

In [ ]:
benchmark = []
for b in tqdm(range(50)):
    sm = Salesman(12, 300, start=0, elite=.5, mutation=.3, generate=False)
    benchmark.append(sm.run(1000))
print(*benchmark, sep="\n")

In [4]:
print(min([b[1] for b in benchmark]))


33
